In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# regular imports
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
%matplotlib inline

import albumentations as A
from albumentations.pytorch import ToTensorV2

# Lightning import
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
# from pl_bolts.datamodules.mnist_datamodule import MNISTDataModule

# PyTorch imports
import torch
from torch import nn
from torch.nn import functional as F
print(f'Cuda available: {torch.cuda.is_available()}')

import wandb
wandb.login()

# internal imports
from src.callbacks import ImagePredictionLogger
from src.dataset import MNISTDataModule
from src.models import CNN, LitModel
from src.utils import sweep_iteration

---

In [ ]:
# MODEL_CKPT_PATH = '../model/'
# MODEL_CKPT = '../model/model-{epoch:02d}-{val_loss:.2f}'

# checkpoint_callback = ModelCheckpoint(
#     monitor='val_loss',
#     filepath=MODEL_CKPT ,
#     save_top_k=3,
#     mode='min'
# )

In [ ]:
AUG_KWARGS = dict(border_mode=A.cv2.BORDER_CONSTANT, value=0,
                  interpolation=A.cv2.INTER_LANCZOS4)

train_transforms = A.Compose([
    #A.RandomCrop(width=24, height=24),
    #A.HorizontalFlip(p=0.5),
    #A.GridDistortion(p=0.5, distort_limit=.3, **AUG_KWARGS),
    A.ElasticTransform(p=0.5, sigma=1, alpha=3, alpha_affine=0, **AUG_KWARGS),
    A.ElasticTransform(p=0.5, sigma=1, alpha=1, alpha_affine=3, **AUG_KWARGS),
    A.ShiftScaleRotate(p=1.0, scale_limit=.2, rotate_limit=0, **AUG_KWARGS),
    A.ShiftScaleRotate(p=1.0, scale_limit=0, rotate_limit=25, **AUG_KWARGS),
    #A.CoarseDropout(p=1.0, max_holes=8, max_height=4, max_width=4,
    #                min_holes=1, min_height=4, min_width=4),
    #A.RandomBrightnessContrast(p=0.2),
    #A.Blur(blur_limit=4),
    A.Normalize(mean=(0.0,), std=(1,)),  # , max_pixel_value=255),
    ToTensorV2()
])

In [ ]:
proj = 'SimSiam-Lightning'

# Setup datamodule. Comes with its own train / val / test dataloader.
mnist = MNISTDataModule('../data/', batch_size=512, train_transforms=train_transforms)
mnist.prepare_data()
mnist.setup()

cnn = CNN(C=mnist.dims[0], num_classes=mnist.num_classes)  # backbone
metrics = [('acc', pl.metrics.Accuracy()),]  # (name for log entry, metric)
model = LitModel(
    datamodule=mnist, backbone=cnn, metrics=metrics,
    lr=1e-3, flood=True
)

In [ ]:
wandb_logger = WandbLogger(project=proj, job_type='train')
callbacks = [
    LearningRateMonitor(),  # log the LR
    ImagePredictionLogger(mnist.val_dataloader(batch_size=5000), n_samples=32),
]

trainer = Trainer(
    max_epochs=200, #gpus=-1,  # all GPUs
    logger=wandb_logger, callbacks=callbacks,
    accumulate_grad_batches=1, gradient_clip_val=0,  # 0.5
    progress_bar_refresh_rate=20,
    #checkpoint_callback=checkpoint_callback
    fast_dev_run=True,
)

In [ ]:
trainer.fit(model, mnist)

In [ ]:
trainer.test()

In [ ]:
wandb.finish()

---

# LR finder

In [ ]:
# # Learning rate finder
# lr_finder = trainer.tuner.lr_find(model, num_training=3000, mode='linear', max_lr=1e-2)
# # lr_finder.results  # Results can be found in
# fig = lr_finder.plot(suggest=True)
# lr_finder.suggestion()
# model.hparams.lr = new_lr  # update hparams of the model

# Hyperparameter sweep 

In [ ]:
# from src.sweeps import sweep_config

# sweep_id = wandb.sweep(sweep_config, project=proj)

In [ ]:
# wandb.agent(sweep_id, function=sweep_iteration, project=proj)